### CS4102 - Geometric Foundations of Data Analysis I
Prof. Götz Pfeiffer<br />
School of Mathematical and Statistical Sciences<br />
University of Galway

# Week 8

##  Special methods

* Each python class is aware of certain **special methods**.
* In a user defined class, they can be overwritten to support standard behavior for its instances.

### List-like behavior

* For example, here is a class `MyList` of objects that should **behave like lists**.
* Each such object stores a position `pos` (in `range(10)`).
* It should then behave like the standard basis vector that has all its entries $0$, except for an entry $1$ in position `pos`.
* So the special constructor method `__init__` is defined with `pos` as argument, and it stores whatever is passed in the `pos` component of the new object.

In [ ]:
class MyList:
    def __init__(self, pos):
        self.pos = pos

* We can now construct such an object and print it.

In [ ]:
l = MyList(5)
l.pos

In [ ]:
l

In [ ]:
l.__class__.__name__

* Wouldn't it be nice if that string representation of a `MyList` object was a bit more informative, like the expression that was used to create it in the first place?
* We can arrange that in the special method `__repr__`

In [ ]:
class MyList:
    def __init__(self, pos):
        self.pos = pos
        
    def __repr__(self):
        return f"{self.__class__.__name__}({self.pos})"

In [ ]:
l = MyList(4)
l

In [ ]:
# len(l)

* When the special method `__len__` is implemented, a `MyList` object can respond to a `len` call.

In [ ]:
class MyList:
    def __init__(self, pos):
        self.pos = pos
        
    def __repr__(self):
        return f"MyList({self.pos})"
    
    def __len__(self):
        return 10

In [ ]:
l = MyList(6)
len(l)

In [ ]:
l.pos

In [ ]:
# l[6]

* An implementation of the special method `__getitem__` makes `MyList` objects subscriptable.

In [ ]:
class MyList:
    def __init__(self, pos):
        self.pos = pos
        
    def __repr__(self):
        return f"MyList({self.pos})"
    
    def __len__(self):
        return 10
    
    def __getitem__(self, i):
        return 1 if i == self.pos else 0

In [ ]:
l = MyList(8)
l[8]

In [ ]:
l[1]

In [ ]:
[l[i] for i in range(len(l))]

In [ ]:
# list(l)

* Implementing an iterator ...

In [ ]:
class MyList:
    def __init__(self, pos):
        self.pos = pos
        
    def __repr__(self):
        return f"MyList({self.pos})"
    
    def __len__(self):
        return 10
    
    def __getitem__(self, i):
        #return 1 if i == self.pos else 0
        return int(i == self.pos)

    def __iter__(self):
        return iter(self[i] for i in range(len(self)))


In [ ]:
int(False)

In [ ]:
l = MyList(4)
l[4]

In [ ]:
l[2]

In [ ]:
list(MyList(3))

* But, assignment to positions is not possible.  What should it mean after all ...

In [ ]:
# l[4] = 1

### Function like behavior

* In a similar way, objects can be taught to act like functions, i.e., be callable.
* For this, we implement the special method `__call__`.

* For example, if we want a `MyList` object to act like the function that takes a numerical argument $c$, and returns the $c$-multiple of the standard basis vector it represents:

In [ ]:
class MyList:
    def __init__(self, pos):
        self.pos = pos
        
    def __repr__(self):
        return f"MyList({self.pos})"
    
    def __len__(self):
        return 10
    
    def __getitem__(self, i):
        return 1 if i == self.pos else 0
    
    def __iter__(self):
        return iter(self[i] for i in range(len(self)))
        
    def __call__(self, c):
        return [c*x for x in self]

In [ ]:
l = MyList(5)
l(3)

In [ ]:
MyList(1)(8)

## Inheritance

* A class can inherit (methods) from another class
* Instances of the following class `BetterList` will have all the functionality of `MyList` objects ...
* ... and additionally they can assign to positions.

In [ ]:
class BetterList(MyList):
    def __setitem__(self, i, val):
        self.pos = i

In [ ]:
m = BetterList(3)
m

In [ ]:
m[0] = 1

In [ ]:
list(m)

In [ ]:
m

* To fix the string representation, we can overload the `__repr__` method in `BetterList`.
* This change will not affect `MyList` objects.

In [ ]:
class BetterList(MyList):
    def __setitem__(self, i, val):
        self.pos = i

    def __repr__(self):
        return f"BetterList({self.pos})"

In [ ]:
m = BetterList(7)
m

In [ ]:
l = MyList(7)
l

## Abstract Classes

* Sometimes it can be useful to define an abstract class, i.e., a class that is not meant to be instantiated.
* No objects of this type would ever be created.
* The class rather serves as a prototype for a collection of (concrete) classes that share some common behaviour.

In [ ]:
import os
import numpy as np
from PIL import Image

def read_images(root):
    c = 0
    X, y = [], []
    for folder in next(os.walk(root))[1]:
        for name in os.listdir(os.path.join(root, folder)):
            path = os.path.join(root, folder, name)
            im = Image.open(path)
            X.append(np.array(im))
            y.append(c)
        c += 1
    return np.array(X), y

root = "orl_faces"
X, y = read_images(root)
X.reshape((X.shape[0], -1))

* Here is an example of an abstract class for a yet to be defined collection of different notions of distance.
* It consists of a constructor, a string representation, and an implementation of `__call__` that just raises an error.

In [ ]:
class Distance:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return self.name

    def __call__(self, p, q):
        raise NotImplementedError(f"don't know yet how to {self}(p, q).")

In [ ]:
d = Distance("d")
d

In [ ]:
# d(1,2)

### Euclidean Distance

* Eulidean distance between $x$ and $y$ is
\\[
 e(x, y) = (\sum_i (x_i - y_i)^2)^{1/2}
\\]

* To implement this, we could literally take the above `Distance` class as a blueprint, copy and paste most of it and provide a working implementation of `__call__`:

In [ ]:
class EuclideanDist:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return self.name

    def __call__(self,p,q):
        p = np.array(p).flatten()
        q = np.array(q).flatten()
        return np.sqrt(np.sum((p - q)**2))

* Then we make an object `e` of this type, name it `"e"` so that it prints itself as `e` and the call it with two vectors as arguments.

In [ ]:
e = EuclideanDist("e")
e

In [ ]:
e([1,2,3], [4,5,6])

* Through the clever use of numpy's `flatten` method, we can als measure distance between images from the database imported above.
* Images are $2$-dimensional arrays, which can be regarded as $1$-dimensional vectors.

In [ ]:
e(X[1], X[2])

In [ ]:
e(X[1], X[21])

* But inheritance is more efficient than copy-and-paste.
* If we define `EuclideanDist` as subclass of `Distance`, we only need to define the `__call__` method.
* `__init__` and `__repr__` are inherited.

In [ ]:
class EuclideanDist(Distance):
    def __call__(self, p, q):
        p = np.array(p).flatten()
        q = np.array(q).flatten()
        return np.sqrt(np.sum((p - q)**2))

In [ ]:
e = EuclideanDist("e")
e

In [ ]:
e([1,2,3], [4,5,6])

* The Taxicab distance between vectors $x$ and $y$ is:
\\[
   t(x, y) = \sum_i |x_i - y_i |
\\]
* Again, this notion of distance can be defined as a subclass of the abstract `Distance` class by simply implementing the formula as part of the `__call__` method.

In [ ]:
class TaxicabDist(Distance):
    def __call__(self, p, q):
        p = np.array(p).flatten()
        q = np.array(q).flatten()
        return np.sum(np.abs(p - q))   

* Now we can make and use an object of this type.

In [ ]:
t = TaxicabDist("t")
t

In [ ]:
t([1,2,3],[4,5,6])

In [ ]:
t(X[1], X[2])

In [ ]:
t(X[1], X[21])

* The Infintity Distance between vectors $x$ and $y$ is:
\\[
  i(x, y) = \max |x_i - y_i|
\\]
* As a subclass of `Distance`:

In [ ]:
class InfinityDist(Distance):
    def __call__(self, p, q):
        p = np.array(p).flatten()
        q = np.array(q).flatten()
        return np.max(np.abs(p - q))   

In [ ]:
i = InfinityDist("i")
i

In [ ]:
i([1,2,3],[4,5,6])

In [ ]:
i(X[1], X[2])

In [ ]:
i(X[1], X[21])

* Did it say above that an abstract class is the place to put common code?
* We could define this collection of classes even more succintly, by defining the operation of flattening the arguments, which is common to all three concrete distance classes, into the abstract class:

In [ ]:
class Distance:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return self.name

    def __call__(self, p, q):
        raise NotImplementedError(f"don't know yet how to {self}(p, q).")
        
    def flatten(self, p, q):
        p = np.array(p).flatten()
        q = np.array(q).flatten()
        return p - q

* Then all the subclasses essentially are one-liners ...

In [ ]:
class EuclideanDist(Distance):
    def __call__(self, p, q):
        return np.sqrt(np.sum(self.flatten(p, q)**2))

In [ ]:
e = EuclideanDist("e")
e([1,2,3],[4,5,6])

In [ ]:
class TaxicabDist(Distance):
    def __call__(self, p, q):
        return np.sum(np.abs(self.flatten(p, q)))   

In [ ]:
t = TaxicabDist("t")
t([1,2,3],[4,5,6])

In [ ]:
class InfinityDist(Distance):
    def __call__(self, p, q):
        return np.max(np.abs(self.flatten(p, q)))   

In [ ]:
i = InfinityDist("i")
i([1,2,3],[4,5,6])